In [1]:
%pwd

'c:\\Users\\Rizen3\\Desktop\\Vamshi\\Machine Learning and DSA\\SelfProjects\\medic.ai\\research'

In [11]:
import os
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls = PyPDFLoader)
    docs = loader.load()
    return docs

In [16]:
extracted_data = load_pdf(data='..//Data/')

Error loading file ..\Data\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf


ImportError: pypdf package not found, please install it with `pip install pypdf`

In [20]:
# extracted_data

# Chunking : 
def text_split(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(data)
    return text_chunks

text_chunks = text_split(extracted_data)
print(f"length of chunks : {len(text_chunks)}")

length of chunks : 7152


In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings 

def download_hf_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hf_embeddings()
print(embeddings)

C:\Users\Rizen3\AppData\Local\Temp\ipykernel_10716\1856694351.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Rizen3\Desktop\Vamshi\Machine Learning and DSA\SelfProjects\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Rizen3\Desktop\Vamshi\Machine Learning and DSA\SelfProjects\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarn

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [23]:
test = embeddings.embed_query("hey there")
print(f"Length : {len(test)}") # 384 for 384 dimensions

Length : 384


In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key = PINECONE_API_KEY)

index_name = "medicoai"

pc.create_index(
    name=index_name,
    dimension= 384, 
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '52c4d8d9be1b2c38ef569f04cd3f127c', 'Date': 'Fri, 11 Oct 2024 05:56:22 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [19]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name= index_name,
    embedding= embeddings
)


NameError: name 'text_chunks' is not defined

In [ ]:
# Above cell will store embeddings of every single chunk of our document.

In [20]:
index_name = "medicoai"
pc = Pinecone(api_key = PINECONE_API_KEY)
docsearch = pc.from_existing_index(
    name=index_name,
    dimension= 384, 
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)
docsearch


AttributeError: 'PineconeGRPC' object has no attribute 'from_existing_index'